# EE4375-2022: Extension of Sixth Lab Session: Quadrature and Second Order Lagrange Elements
Extension of 2D FEM using second order elements and quadrature. 

## Import Packages

In [ ]:
using LinearAlgebra
using FastGaussQuadrature
using Plots
using SparseArrays
using BenchmarkTools

## Section 1/: Introduction  

## Section 2:/ Quadrature over Triangles 
Functions taken from [MinFEM.jl](https://github.com/MinFEM/MinFEM.jl/blob/master/src/quadrature.jl)

In [1]:
"""
    gausslegendre_points(order::Int64) -> Array{Float64,1}
    
Returns coordinates of the Gauss-Legendre quadrature points on the default interval [-1,1] 
for exact integration of polynomials up to the given order.
"""
function gausslegendre_points(order::Int64)
    if order <= 1
        return [0]
    elseif order <= 3
        val = 1/sqrt(3)
        return [-val, val]
    elseif order <= 5
        val = sqrt(3/5)
        return  [-val, 0, val]
    elseif order <= 7
        val = 2/7 * sqrt(6/5)
        val1 = sqrt(3/7 - val)
        val2 = sqrt(3/7 + val)
        return  [-val2, -val1, val1, val2]
    elseif order <= 9
        val = 2 * sqrt(10/7)
        val1 = 1/3 * sqrt(5 - val)
        val2 = 1/3 * sqrt(5 + val)
        return  [-val2, -val1, 0, val1, val2]
    else
        throw(ErrorException("Order $order not possible. " *
                                "Highest possible order for 1D is 9."))
    end
end

gausslegendre_points

In [3]:
gausslegendre_points(5)

3-element Vector{Float64}:
 -0.7745966692414834
  0.0
  0.7745966692414834

In [4]:
"""
    gausslegendre_weights(order::Int64) -> Array{Float64,1}
    
Returns weights of the Gauss-Legendre quadrature points on the default interval [-1,1] 
for exact integration of polynomials up to the given order.
"""
function gausslegendre_weights(order::Int64)
    if order <= 1
        return [2]
    elseif order <= 3
        return [1, 1]
    elseif order <= 5
        val = 5/9
        return  [val, 8/9, val]
    elseif order <= 7
        val = sqrt(30)
        val1 = (18 + val) / 36
        val2 = (18 - val) / 36
        return  [val2, val1, val1, val2]
    elseif order <= 9
        val = 13 * sqrt(70)
        val1 = (322 + val) / 900
        val2 = (322 - val) / 900
        return  [val2, val1, 128/225, val1, val2]
    else
        throw(ErrorException("Order $order not possible. " *
                                "Highest possible order for 1D is 9."))
    end
end

gausslegendre_weights

In [5]:
gausslegendre_weights(3)

2-element Vector{Int64}:
 1
 1

In [6]:
"""
    compute_coordinates_triangle(order::Int64) -> Array{Array{Float64,1},1}
    
Returns coordinates of the Gauss-Legendre quadrature points 
on the 2-dimensional FEM reference element 
for exact integration of polynomials up to the given order.
"""
function compute_coordinates_triangle(order::Int64)
    order1 = order + 1
    p1 = gausslegendre_points(order1)
    n = length(p1)

    c = Array{Array{Float64,1},1}(undef, n*n)

    r = 1
    for i = 1:n
        for j = 1:n
            c[r] = [(1 + p1[i]) / 2 , (1 - p1[i]) * (1 + p1[j]) / 4]
            r = r + 1    
        end
    end
    return c
end

compute_coordinates_triangle

In [8]:
compute_coordinates_triangle(7)

25-element Vector{Vector{Float64}}:
 [0.04691007703066802, 0.04470952170364481]
 [0.04691007703066802, 0.21994012483967862]
 [0.04691007703066802, 0.47654496148466596]
 [0.04691007703066802, 0.7331497981296533]
 [0.04691007703066802, 0.9083804012656871]
 [0.2307653449471585, 0.036084856923188136]
 [0.2307653449471585, 0.17751270051857745]
 [0.2307653449471585, 0.38461732752642075]
 [0.2307653449471585, 0.591721954534264]
 [0.2307653449471585, 0.7331497981296533]
 [0.5, 0.02345503851533401]
 [0.5, 0.11538267247357925]
 [0.5, 0.25]
 [0.5, 0.38461732752642075]
 [0.5, 0.47654496148466596]
 [0.7692346550528415, 0.010825220107479883]
 [0.7692346550528415, 0.053252644428581054]
 [0.7692346550528415, 0.11538267247357925]
 [0.7692346550528415, 0.17751270051857745]
 [0.7692346550528415, 0.21994012483967862]
 [0.9530899229693319, 0.002200555327023207]
 [0.9530899229693319, 0.010825220107479883]
 [0.9530899229693319, 0.02345503851533401]
 [0.9530899229693319, 0.036084856923188136]
 [0.953089922969

In [9]:
"""
    compute_weights_triangle(order::Int64) -> Array{Float64,1}
    
Returns weights of the Gauss-Legendre quadrature points 
on the 2-dimensional FEM reference element 
for exact integration of polynomials up to the given order.
"""
function compute_weights_triangle(order::Int64)
    order1 = order + 1    
    p1 = gausslegendre_points(order1)
    w1 = gausslegendre_weights(order1)
    n = length(p1)
    
    w = Array{Float64,1}(undef, n*n)

    r = 1
    for i = 1:n
        for j = 1:n
            w[r] = (1 - p1[i]) / 8 * w1[i] * w1[j]
            r = r + 1    
        end
    end
    return w
end

compute_weights_triangle

In [11]:
compute_weights_triangle(5)

16-element Vector{Float64}:
 0.02815038307692565
 0.0527752773542295
 0.0527752773542295
 0.02815038307692565
 0.03799714764795021
 0.0712356204997401
 0.0712356204997401
 0.03799714764795021
 0.018715815315012756
 0.0350877052529335
 0.0350877052529335
 0.018715815315012756
 0.0021003652444748482
 0.003937685608733464
 0.003937685608733464
 0.0021003652444748482